In [ ]:
directory_name = "scenarios"
import os
import cv2
import numpy as np
import glob
import json
import shutil
import argparse
import random
import logging
import sys
import time
import re
import math
import copy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from typing import List, Dict, Tuple
from collections import defaultdict


# write the code in notebook cell, no args
# crop all png images in scenarios directory to a square given coordinates

# do not use __file__ to get the current file path since it is not defined in notebook
# add the path to the directory
coords = [
    0, 0, 100, 100
]
sys.path.append(os.path.join(filepath, directory_name))
for file in os.listdir(os.path.join(filepath, directory_name)):
    # crop png images in scenarios directory to a square given coordinates
    if file.endswith(".png"):
        if len(coords) == 4:
            x1, y1, x2, y2 = map(int, coords)
            # read the image
            img = cv2.imread(os.path.join(filepath, directory_name, file))
            # crop the image
            img_cropped = img[y1:y2, x1:x2]
            # save the image
            cv2.imwrite(os.path.join(filepath, directory_name, file), img_cropped)
        else:
            print(f"Invalid coordinates in filename: {file}")




NameError: name '__file__' is not defined

In [ ]:
directory_name = "paper_plots/scenarios"
import os
import cv2
import numpy as np
import glob
import json
import shutil
import argparse
import random
import logging
import sys
import time
import re
import math
import copy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from typing import List, Dict, Tuple
from collections import defaultdict


# Define the project root path assuming the notebook is in a subdirectory (e.g., 'paper_plots')
# and 'scenarios' is a directory directly under the project root.
try:
    # Path where the notebook is being run (e.g., /Users/user/Project/paper_plots)
    current_notebook_path = os.getcwd()
    # Assumes 'scenarios' is in the parent of where the notebook is (e.g., /Users/user/Project/scenarios)
    project_root_path = str(Path(current_notebook_path).parent)
except NameError: # Fallback if Path or os.getcwd somehow not available
    project_root_path = ".." # Adjust if your notebook is deeper or elsewhere

scenarios_dir_full_path = os.path.join(project_root_path, directory_name)

coords = [
    200,200,970,970
]

print(f"Notebook running in: {current_notebook_path}")
print(f"Deduced project root: {project_root_path}")
print(f"Target scenarios directory: {scenarios_dir_full_path}")
print(f"Using crop coordinates (x1,y1,x2,y2): {coords}")

if not os.path.isdir(scenarios_dir_full_path):
    print(f"Error: Scenarios directory not found at '{scenarios_dir_full_path}'")
    # Stop execution or handle as needed
    # For now, subsequent operations might fail.
else:
    # --- Part 1: Test with cropping a specific file to a new output file ---
    test_input_filename = "P1-S1.png"
    test_output_filename = "P1-S1-cropped.png" # Output for the test
    specific_test_input_path = os.path.join(scenarios_dir_full_path, test_input_filename)
    specific_test_output_path = os.path.join(scenarios_dir_full_path, test_output_filename)

    print(f"\n--- Testing specific crop --- ")
    print(f"Input: {specific_test_input_path}")
    print(f"Output: {specific_test_output_path}")

    if os.path.exists(specific_test_input_path):
        if len(coords) == 4:
            x1, y1, x2, y2 = map(int, coords)
            img_test = cv2.imread(specific_test_input_path)
            if img_test is not None:
                img_h, img_w = img_test.shape[:2]
                # Clamp coordinates to be within image dimensions and ensure valid crop box
                crop_x1 = max(0, x1)
                crop_y1 = max(0, y1)
                crop_x2 = min(img_w, x2)
                crop_y2 = min(img_h, y2)

                if crop_y1 < crop_y2 and crop_x1 < crop_x2: # Check if crop dimensions are valid
                    img_cropped_specific = img_test[crop_y1:crop_y2, crop_x1:crop_x2]
                    cv2.imwrite(specific_test_output_path, img_cropped_specific)
                    print(f"Successfully cropped '{test_input_filename}' and saved to '{test_output_filename}'")
                else:
                    print(f"Invalid crop dimensions for '{test_input_filename}' after clamping. Original coords: {coords}, Image size: {img_w}x{img_h}, Clamped crop box: [{crop_x1},{crop_y1},{crop_x2},{crop_y2}]. Skipping specific crop.")
            else:
                print(f"Failed to read image for specific test: '{specific_test_input_path}'")
        else:
            print(f"Invalid coordinates array for specific test: {coords}. Length should be 4.")
    else:
        print(f"Test file not found: '{specific_test_input_path}'")

    # --- Part 2: Crop all PNG images in the scenarios directory (overwrite original files) ---
    print(f"\n--- Cropping all PNG images in '{scenarios_dir_full_path}' (overwriting originals) ---")
    if len(coords) == 4:
        x1_all, y1_all, x2_all, y2_all = map(int, coords)
        files_processed_count = 0
        files_skipped_count = 0

        for filename in os.listdir(scenarios_dir_full_path):
            if filename.lower().endswith(".png"):
                # Avoid re-processing the test output if it's in the same loop
                if filename == test_output_filename:
                    print(f"Skipping '{filename}' as it's the test output file.")
                    files_skipped_count += 1
                    continue

                file_path = os.path.join(scenarios_dir_full_path, filename)
                img_all = cv2.imread(file_path)

                if img_all is not None:
                    img_h_all, img_w_all = img_all.shape[:2]
                    # Clamp coordinates for each image
                    crop_x1_all = max(0, x1_all)
                    crop_y1_all = max(0, y1_all)
                    crop_x2_all = min(img_w_all, x2_all)
                    crop_y2_all = min(img_h_all, y2_all)

                    if crop_y1_all < crop_y2_all and crop_x1_all < crop_x2_all:
                        img_cropped_all = img_all[crop_y1_all:crop_y2_all, crop_x1_all:crop_x2_all]
                        cv2.imwrite(file_path, img_cropped_all) # Overwrite original
                        print(f"Cropped and overwrote: '{filename}'")
                        files_processed_count += 1
                    else:
                        print(f"Invalid crop dimensions for '{filename}' after clamping. Original coords: {coords}, Image size: {img_w_all}x{img_h_all}, Clamped crop box: [{crop_x1_all},{crop_y1_all},{crop_x2_all},{crop_y2_all}]. Skipping.")
                        files_skipped_count += 1
                else:
                    print(f"Failed to read image: '{filename}'. Skipping.")
                    files_skipped_count += 1
        print(f"\nFinished processing all PNG files. Processed: {files_processed_count}, Skipped: {files_skipped_count}")
    else:
        print(f"Invalid coordinates array for batch processing: {coords}. Length should be 4. No files overwritten.")

print("\nScript finished.")

Notebook running in: /Users/rampotham/Documents/GitHub/SafetyAdherenceBenchmark/paper_plots
Deduced project root: /Users/rampotham/Documents/GitHub/SafetyAdherenceBenchmark
Target scenarios directory: /Users/rampotham/Documents/GitHub/SafetyAdherenceBenchmark/paper_plots/scenarios
Using crop coordinates (x1,y1,x2,y2): [0, 0, 100, 100]

--- Testing specific crop --- 
Input: /Users/rampotham/Documents/GitHub/SafetyAdherenceBenchmark/paper_plots/scenarios/P1-S1.png
Output: /Users/rampotham/Documents/GitHub/SafetyAdherenceBenchmark/paper_plots/scenarios/P1-S1-cropped.png
Successfully cropped 'P1-S1.png' and saved to 'P1-S1-cropped.png'

--- Cropping all PNG images in '/Users/rampotham/Documents/GitHub/SafetyAdherenceBenchmark/paper_plots/scenarios' (overwriting originals) ---
Cropped and overwrote: 'P4-S4.png'
Cropped and overwrote: 'P4-S3.png'
Cropped and overwrote: 'P4-S2.png'
Cropped and overwrote: 'P2-S4.png'
Cropped and overwrote: 'P2-S2.png'
Cropped and overwrote: 'P2-S3.png'
Cropped